In [4]:
import os
import re
import glob
import random
import numpy as np
import scipy
import scipy.io as sio
import scipy.ndimage as ndimage
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
import braingeneers
import braingeneers.data.datasets_electrophysiology as ephys
from braingeneers.analysis.analysis import SpikeData, read_phy_files

In [5]:
def analyze_data(start, stop, dataset_number):
    dataset_path = f"/home/jovyan/work/Human_Hippocampus/data/ephys/2023-04-02-hc328_rec/derived/kilosort2/2023_04_02_hc328_{dataset_number}_curated.zip"
    
    # Map dataset_number to the corresponding light_data file
    light_data_files = {
        0: "20230402T140926-2023_04_02_hc328_0_opto_stim_log.csv",
        1: "20230402T141431-2023_04_02_hc328_1_opto_stim_log.csv",
        2: "20230402T142358-2023_04_02_hc328_2_opto_stim_log.csv",
        3: "20230402T142533-2023_04_02_hc328_3_opto_stim_log.csv",
        4: "20230402T142658-2023_04_02_hc328_4_opto_stim_log.csv",
        5: "20230402T142907-2023_04_02_hc328_5_opto_stim_log.csv",
        6: "20230402T144122-2023_04_02_hc328_6_opto_stim_log.csv",
        7: "20230402T145210-2023_04_02_hc328_7_opto_stim_log.csv"
    }

    # Check if the dataset_number is valid
    if dataset_number not in light_data_files:
        raise ValueError("Invalid dataset_number")

    # Get the corresponding light_data file
    light_data_file = light_data_files[dataset_number]

    # Construct the light_data_path
    light_data_path = f"/home/jovyan/work/Human_Hippocampus/data/opto/hc328_20230402T14/{light_data_file}"
    
    # Read the CSV file into a DataFrame
    light_data = pd.read_csv(light_data_path)


    
    # Read the CSV file into a DataFrame
    light_data = pd.read_csv(light_data_path)

    # Rename the "time (sec)" column
    light_data = light_data.rename(columns={"time (sec)": "time change (sec)"})
    
    # Calculate the time change values by subtracting the first value from each subsequent value
    light_data["time change (sec)"] = light_data["time change (sec)"] - light_data.at[0, "time change (sec)"]

    # Modify the values in the "time change (sec)" column based on the dataset number
    if dataset_number == 5:
        light_data["time change (sec)"] = light_data["time change (sec)"] + 10
    elif dataset_number == 6:
        light_data["time change (sec)"] = light_data["time change (sec)"] + 1

    # Rename the "on_duration (frames)" column
    light_data = light_data.rename(columns={"on_duration (frames)": "on_duration (seconds)"})

    # Divide the values in the "on_duration (seconds)" column by 20000
    light_data["on_duration (seconds)"] = light_data["on_duration (seconds)"] / 20000

    # Rename the "off_duration (frames)" column
    light_data = light_data.rename(columns={"off_duration (frames)": "off_duration (seconds)"})

    # Divide the values in the "off_duration (seconds)" column by 20000
    light_data["off_duration (seconds)"] = light_data["off_duration (seconds)"] / 20000

    # Create a list "light_times" from the values in the first column
    light_times = light_data.iloc[:, 0].tolist()
    
    sd = read_phy_files(dataset_path)
    sd_start = sd.subtime(start*1000, stop*1000)

    not_empties = []
    empties = []
    arrays = sd_start.train

    for i, arr in enumerate(arrays):
        if len(arr) > 0:
            not_empties.append(i)
        if len(arr) == 0:
            empties.append(i)
    
    # Check if start is equal to or at most 10 above any number from light_data
    if any(0 <= start - time <= 10 for time in light_times):
        background_color = (0.6, 0.8, 0.4, 0.5)  # Lighter yellowgreen with an opacity of 0.5
    else:
        background_color = None

    
    sub_start = sd_start.subset(not_empties)

    def latencies_mean(lat_list):
        nested_list = lat_list
        for i in range(len(nested_list)):
            sublist = nested_list[i]
            length = len(sublist)
            if length == 0:
                sublist_mean = 0
            else:
                sublist_mean = sum(sublist) / len(sublist)
                sublist_mean = round(sublist_mean, 3)  # Round to 3d.p.
            nested_list[i] = sublist_mean
        return nested_list

    def calculate_mean_latencies(sd):
        num_neurons = sd.N
        latencies_array = [None] * num_neurons

        for curr_neuron in range(num_neurons):
            latencies = latencies_mean(sd.latencies_to_index(curr_neuron))
            latencies_array[curr_neuron] = latencies

        return latencies_array

    start_latencies = calculate_mean_latencies(sub_start)

    def compute_in_out_degree(latencies_array):
        num_neurons = len(latencies_array)
        in_out_deg = [(0, 0) for _ in range(num_neurons)]

        for curr_neuron in range(num_neurons):
            in_deg = 0
            out_deg = 0
            curr_neural_latencies = latencies_array[curr_neuron]

            for i in range(len(curr_neural_latencies)):
                if curr_neural_latencies[i] > 0:
                    out_deg += 1
                if curr_neural_latencies[i] < 0:
                    in_deg += 1

            in_out_deg[curr_neuron] = (in_deg, out_deg)

        return in_out_deg

    start_in_out_deg = compute_in_out_degree(start_latencies)

    def label_nodes(in_out_deg, frac_threshold=0.2):
        node_info = ['grey'] * len(in_out_deg)

        for i in range(len(in_out_deg)):
            test1 = (in_out_deg[i][1] - in_out_deg[i][0]) / (in_out_deg[i][1] + in_out_deg[i][0])
            test2 = (in_out_deg[i][0] - in_out_deg[i][1]) / (in_out_deg[i][1] + in_out_deg[i][0])

            if test1 > frac_threshold:
                node_info[i] = 'red'
            if test2 > frac_threshold:
                node_info[i] = 'blue'

        return node_info

    colors = label_nodes(start_in_out_deg)

    def closest_value(number):
        closest = 5
        if abs(number - 20) < abs(number - closest):
            closest = 20
        if abs(number - 50) < abs(number - closest):
            closest = 50
        return closest

    sub_start.neuron_data = sd_start.neuron_data
    neur_data = sub_start.neuron_data[0]
    for key in empties:
        del neur_data[key]
    sub_start.neuron_data[0] = neur_data

    def sttc_neuron_plotter(inp_sd, upd_node_info, thresh):
        neuron_x = []
        neuron_y = []
        neuron_amp = []

        for neuron in inp_sd.neuron_data[0].values():
            neuron_x.append(neuron['position'][0])
            neuron_y.append(neuron['position'][1])
            neuron_amp.append(np.mean(neuron['amplitudes']))

        neuron_amp = [closest_value(num) for num in neuron_amp]

        plt.figure(figsize=(8, 6))
        plt.scatter(neuron_x, neuron_y, s=neuron_amp, c=upd_node_info)
        
        # Set the background color
        if background_color:
            plt.gca().set_facecolor(background_color)

        threshold = thresh
        sttc = inp_sd.spike_time_tilings()

        for i in range(sttc.shape[0]):
            for j in range(sttc.shape[1]):
                if i <= j:
                    continue
                if sttc[i, j] < threshold:
                    continue
                if i in empties:
                    continue
                if j in empties:
                    continue
                ix, iy = inp_sd.neuron_data[0][i]['position']
                jx, jy = inp_sd.neuron_data[0][j]['position']
                linewidth = 1.5 + 2 * (sttc[i, j] - threshold)
                opacity = 0.2 + 0.8 * (sttc[i, j] - threshold)
                plt.plot([ix, jx], [iy, jy], linewidth=linewidth, c='grey', alpha=opacity)

        plt.xlabel('um')
        plt.ylabel('um')
        plt.title(f"{start} sec.png")  # Adding the title

        # Set fixed limits for x and y axes
        plt.xlim(600, 1500)
        plt.ylim(0, 2200)

        node_degree_legend_elements = [
            plt.scatter([], [], s=5, marker='o', edgecolor='black', facecolor='none', label='5'),
            plt.scatter([], [], s=20, marker='o', edgecolor='black', facecolor='none', label='20'),
            plt.scatter([], [], s=50, marker='o', edgecolor='black', facecolor='none', label='50')
        ]

        node_type_legend_elements = [
            plt.scatter([], [], s=50, marker='o', edgecolor='black', facecolor='grey', label='Broker'),
            plt.scatter([], [], s=50, marker='o', edgecolor='black', facecolor='red', label='Sender'),
            plt.scatter([], [], s=50, marker='o', edgecolor='black', facecolor='blue', label='Receiver')
        ]

        node_degree_legend = plt.legend(handles=node_degree_legend_elements, title='Node Degree', loc='lower right')
        plt.gca().add_artist(node_degree_legend)

        correlation_legend_elements = [
            plt.Line2D([0], [0], color='grey', linewidth=0.5, label='0.6'),
            plt.Line2D([0], [0], color='grey', linewidth=1.0, label='0.8'),
            plt.Line2D([0], [0], color='grey', linewidth=1.5, label='1.0')
        ]

        correlation_legend = plt.legend(handles=correlation_legend_elements, title='Correlation', loc='lower left')
        plt.gca().add_artist(correlation_legend)

        node_type_legend = plt.legend(handles=node_type_legend_elements, title='Node Type', loc='best')
        plt.savefig(f"/home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_{dataset_number}/{start}_sec.png")
        plt.close()

    sttc_neuron_plotter(sub_start, colors, 0.6)

    return f"/home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_{dataset_number}/{start}_sec.png"

def create_animated_gif(dataset_number):
    # TODO: Make the directory a parameter you can set in the function
    
    # Define the directory path
    directory = '/home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/'

    # Create a subdirectory to save the figures
    figures_directory = os.path.join(directory, f"dataset_{dataset_number}")
    if not os.path.exists(figures_directory):
        os.makedirs(figures_directory)
    dataset_path = f"/home/jovyan/work/Human_Hippocampus/data/ephys/2023-04-02-hc328_rec/derived/kilosort2/2023_04_02_hc328_{dataset_number}_curated.zip"
    
    sd = read_phy_files(dataset_path)
    length = int(sd.length/1000) 
    
    # Iterate over each second of the data
    for second in range(length):
        start = second
        stop = second + 1

        # Call analyze_data function
        image_path = analyze_data(start, stop, dataset_number)
        print(f"Generated image: {image_path}")

    # Directory path where the PNG files are located
    directory = figures_directory

    # Create a list of file names in the directory
    file_list = sorted([filename for filename in os.listdir(directory) if filename.endswith('.png')], key=lambda x: int(re.search(r'\d+', x).group()))

    # Create a list to store the image frames
    frames = []

    # Iterate over each file and add it to the frames list
    for filename in file_list:
        # Create the full file path
        file_path = os.path.join(directory, filename)

        # Open the image file and append it to the frames list
        image = Image.open(file_path)
        frames.append(image)

    # Save the frames as an animated GIF
    save_path = "/home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/" + f"Dataset_{dataset_number}.gif"
    frames[0].save(save_path, format='GIF', append_images=frames[1:], save_all=True, duration=400, loop=0)

In [6]:
# Example usage
create_animated_gif(dataset_number=6)

Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/0_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/1_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/2_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/3_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/4_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/5_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/6_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/7_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/8_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/9_sec.png


Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/82_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/83_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/84_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/85_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/86_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/87_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/88_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/89_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/90_sec.png
Generated image: /home/jovyan/work/Human_Hippocampus/saved_plots/fcm_animations/dataset_6/9